# Part3. Make a prediction for test file

In part1  we've explained the feature engineering.
The features that will be used to train are:  

1. Entrance
2. Exit
3. Interval Beginning Time
4. Vehicle Class
5. Vehicle Count

Taget to predict is *Payment Type (Cash or E-ZPass)*


In part2, we've discussed whcih algotithm and hyper parameters will be applied.  

The algorithm is **DecisionTreeClassifier**.  
Hyper parameters are **criterion='gini' and max_depth=21**.


In [1]:
import pandas as pd
train_df = pd.read_csv('cash or e-zpass train.csv')
test_df = pd.read_csv('cash or e-zpass test.csv')

In [2]:
# Check the trainning size
train_size = len(train_df)
train_size

6010967

In [3]:
# Split up the train dataset to features and target
train_y = train_df.iloc[:, -1]
train_x = train_df.drop('Payment Type (Cash or E-ZPass)', axis=1)

In [4]:
from sklearn.preprocessing import LabelEncoder


# Encode E-Zpass to 0, CASH to 1
def label_encode_payment_type(payment_type):
    if payment_type == 'E-ZPass':
        return 0
    elif payment_type == 'CASH':
        return 1
    else:
        raise ValueError('Find a value cannot be parsed')
        
# Reverse the encoded payment type 
# 0 to E-Zpass, 1 to CASH
# Useful when final output
def reverse_payment_type(encoded_payment_type):
    if encoded_payment_type == 0:
        return 'E-ZPass'
    elif encoded_payment_type == 1:
        return 'CASH'
    else:
        raise ValueError('Find a value cannot be parsed')
    
# Encoded target
train_y = train_y.apply(label_encode_payment_type)

In [5]:
labelencoder = LabelEncoder()

# Combine the train/test together, to make sure all values have a consistent label encoding 
concat_df = pd.concat([train_x, test_df])

# In part 1, we've decided to drop Date
concat_df = concat_df.drop('Date', axis=1)

# Encoded other features
concat_df['Entrance'] = labelencoder.fit_transform(concat_df['Entrance'])
concat_df['Exit'] = labelencoder.fit_transform(concat_df['Exit'])
concat_df['Vehicle Class'] = labelencoder.fit_transform(concat_df['Vehicle Class'])

In [6]:
# After label encoded, split up the combined dataset to train and test
train_x = concat_df.iloc[:train_size, :]
test_x = concat_df.iloc[train_size:, :]

In [7]:
from sklearn.tree import DecisionTreeClassifier

# In part2, we've decided to train a decision tree with gini and max_depth set to 21
clf = DecisionTreeClassifier(criterion='gini', max_depth=21)
clf.fit(train_x, train_y)

# After model is trained, we can make a prediction
hyp = clf.predict(test_x)

In [8]:
# Because the predicted result is in 0 and 1 format, 
# we need to transform it back to EZ-Pass and CASH
reversed_payment_type = [reverse_payment_type(t) for t in hyp]

In [9]:
# Make sure the length of test x and y are the same

print(len(test_df))
print(len(test_x))
print(len(reversed_payment_type))

2960627
2960627
2960627


In [10]:
# Then we are able to output it as a file 'prediction.csv'
f = open("prediction.csv", "w")
f.write("Cash or E-ZPass\n")
for t in reversed_payment_type:
    f.write(str(t)+"\n")
f.close()